<a href="https://colab.research.google.com/github/kshitijrajsharma/osm2hdx/blob/main/Query_Raw_Data_API_s3_files_HDX.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import requests
import duckdb
import json
import humanize

In [2]:
FOLDER_PATH='HDX'
BASE_API_URL='https://api-prod.raw-data.hotosm.org/v1'
S3_API_URL = f'{BASE_API_URL}/s3/files/?folder={FOLDER_PATH}&prettify=false'

In [3]:
def stream(API_URL):
    session = requests.Session()
    request = session.get(S3_API_URL, stream=True)
    return request.json()

def get_project_info(key):
  session = requests.Session()
  request = session.get(f'{BASE_API_URL}/s3/get/{key}')
  return request.json()


In [4]:
api_data = stream(S3_API_URL)
with open ('stats.json','w') as file :
  file.write(json.dumps(api_data))

In [7]:
con = duckdb.connect()

In [8]:
con.sql("CREATE TABLE stats AS SELECT * FROM read_json_auto('stats.json', format = 'array');")

In [14]:
con.execute("""ALTER TABLE stats ADD COLUMN dir VARCHAR""")
con.execute("""ALTER TABLE stats ADD COLUMN folder VARCHAR""")
con.execute("""ALTER TABLE stats ADD COLUMN category VARCHAR""")
con.execute("""ALTER TABLE stats ADD COLUMN geom_type VARCHAR""")

In [15]:
con.execute("""
  UPDATE stats
  SET
    dir = SPLIT_PART(Key, '/', 1),
    folder = SPLIT_PART(Key, '/', 2),
    category = SPLIT_PART(Key, '/', 3),
    geom_type = SPLIT_PART(Key, '/', 4)
""")
con.commit()

In [25]:
con.sql("""  SELECT folder, MAX(LastModified) AS LatestLastModified
  FROM stats
  GROUP BY folder
  ORDER BY LatestLastModified DESC """)

┌─────────┬─────────────────────┐
│ folder  │ LatestLastModified  │
│ varchar │      timestamp      │
├─────────┼─────────────────────┤
│ NPL     │ 2024-01-24 03:37:00 │
│ VAT     │ 2024-01-24 01:42:28 │
│ MCO     │ 2024-01-24 01:41:30 │
│ SMR     │ 2024-01-24 01:40:37 │
│ BVT     │ 2024-01-24 01:40:10 │
│ BLM     │ 2024-01-24 01:39:39 │
│ LIE     │ 2024-01-24 01:39:26 │
│ SGP     │ 2024-01-24 01:38:46 │
│ AND     │ 2024-01-24 01:38:23 │
│ ABW     │ 2024-01-24 01:37:17 │
│  ·      │          ·          │
│  ·      │          ·          │
│  ·      │          ·          │
│ COD     │ 2024-01-23 08:21:24 │
│ SAU     │ 2024-01-23 08:07:58 │
│ PNG     │ 2024-01-23 08:01:49 │
│ FSM     │ 2024-01-23 07:59:00 │
│ KAZ     │ 2024-01-23 07:58:41 │
│ ARG     │ 2024-01-23 07:58:06 │
│ KIR     │ 2024-01-23 07:43:06 │
│ ITA     │ 2024-01-15 08:24:11 │
│ RUS     │ 2023-12-27 05:32:50 │
│ FRA     │ 2023-12-25 17:27:04 │
├─────────┴─────────────────────┤
│      194 rows (20 shown)      │
└─────────────

In [ ]:
# print(humanize.naturalsize(213754935))
project=get_project_info('TM/hotosm_project_25/meta.json')
print(json.dumps(project, indent=2))